In [ ]:
require 'sqlite3'

db = SQLite3::Database.new 'transactions.db'

In [ ]:
db.execute <<-SQL
DROP TABLE ynab_transactions
SQL

db.execute <<-SQL
CREATE TABLE ynab_transactions (
  id varchar(155),
  date varchar(30),
  payee varchar(255),
  account varchar(255),
  /* Stored as USD cents */
  amount int
)
SQL

In [ ]:
require 'ynab'

access_token = ENV['YNAB_TOKEN']

ynab_api = YNAB::API.new(access_token)

budget_response = ynab_api.budgets.get_budgets
budgets = budget_response.data.budgets

budget_id = ENV['YNAB_BUDGET_ID']

transactions_response = ynab_api.transactions.get_transactions(budget_id, type: 'unapproved')

transactions = transactions_response.data.transactions

transactions.each do |t|
  values = [t.id, t.date.to_s, t.payee_name, t.account_name, t.amount]
        db.execute "insert into ynab_transactions values (?, ?, ?, ?, ?)", values

end
nil

In [ ]:
  db.execute( "select * from ynab_transactions") do |row|
    p row
  end